# 4.10 Customer analysis

# Contents
# 1. Data quality and consistency checks
# 2. Remove redundant columns
# 3. Create new variable - region
# 4. Create exclusion flag
# 5. Customer profiling

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

# def file path
path = r'/Users/alison/Documents/Data Analysis - Tasks/Instacart  Basket Analysis' 

In [3]:
# get latest merged df
all_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'merged_all.pkl'))

# 1. Data quality and consistency checks

In [5]:
# data quality and consistency checks
all_merged.shape

(32404859, 34)

In [7]:
all_merged.columns

Index(['user_id', 'First Name', 'Last_name', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income', 'product_id',
       'product_name', 'aisle_id', 'department_id', 'prices', 'order_id',
       'order_number', 'orders_day_of_week', 'order_hour_of_day',
       'days_since_prior_order', 'New_customer_flag', 'add_to_cart_order',
       'reordered', 'old_merge', 'price_range_loc', 'Busiest_days',
       'busiest period of day', 'max_order', 'loyalty_flag',
       'av_product_price', 'spending_flag', 'Frequent customer',
       'frequency_flag', '_merge'],
      dtype='object')

# 2. Remove redundant columns

In [9]:
# Can remove merge and old_merge columns as not necessary for analysis. 
# Also remove first name and last name for privacy reasons. Can identify by user id. Names are not necessary for the analysis, so it is better to remove them.
all_merged = all_merged.drop(columns=['First Name', 'Last_name', 'old_merge', '_merge'])
print(all_merged)

         user_id  Gender     STATE  Age date_joined  n_dependants fam_status  \
0          26711  Female  Missouri   48    1/1/2017             3    married   
1          26711  Female  Missouri   48    1/1/2017             3    married   
2          26711  Female  Missouri   48    1/1/2017             3    married   
3          26711  Female  Missouri   48    1/1/2017             3    married   
4          26711  Female  Missouri   48    1/1/2017             3    married   
...          ...     ...       ...  ...         ...           ...        ...   
32404854   80148  Female  New York   55    4/1/2020             1    married   
32404855   80148  Female  New York   55    4/1/2020             1    married   
32404856   80148  Female  New York   55    4/1/2020             1    married   
32404857   80148  Female  New York   55    4/1/2020             1    married   
32404858   80148  Female  New York   55    4/1/2020             1    married   

          income  product_id           

In [11]:
all_merged['STATE'].value_counts()

STATE
Pennsylvania            667082
California              659783
Rhode Island            656913
Georgia                 656389
New Mexico              654494
Arizona                 653964
North Carolina          651900
Oklahoma                651739
Alaska                  648495
Minnesota               647825
Massachusetts           646358
Wyoming                 644255
Virginia                641421
Missouri                640732
Texas                   640394
Colorado                639280
Maine                   638583
North Dakota            638491
Alabama                 638003
Kansas                  637538
Louisiana               637482
Delaware                637024
South Carolina          636754
Oregon                  636425
Arkansas                636144
Nevada                  636139
New York                635983
Montana                 635265
South Dakota            633772
Illinois                633024
Hawaii                  632901
Washington              632852
Mi

# 3. Create new variable - region

In [13]:
# organised the states into four offocial regions
result = []
for value in all_merged['STATE']:
    if value in ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']:
        result.append('Northeast')
    elif value in ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']:
        result.append('Midwest')
    elif value in ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana'
]:
        result.append('South')
    else:
        result.append('West')

In [15]:
# create a region column for the above categories
all_merged['Region'] = result

In [17]:
all_merged['Region'].value_counts(dropna=False)

Region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

In [19]:
# cross region with spending flag to find regional spending habits
crosstab = pd.crosstab(all_merged['Region'], all_merged['spending_flag'], dropna = False)

In [20]:
crosstab.to_clipboard()

In [23]:
all_merged.shape

(32404859, 31)

In [25]:
# export new df with Region column
all_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'all_merged.pkl'))

# 4. Create exclusion flag

In [26]:
# create exclusion flag for customers with under 5 orders
# Loop through the order_number to set exclusion_flag
result = []

for value in all_merged['order_number']:
  if value < 5:
    result.append('Yes')
  else:
    result.append('No')

In [27]:
# create new column
all_merged['exclusion_flag'] = result

In [28]:
all_merged['exclusion_flag'].value_counts()

exclusion_flag
No     24414877
Yes     7989982
Name: count, dtype: int64

In [33]:
# Exclude rows where exclusion_flag is "Yes"
new_all_merged = all_merged[all_merged['exclusion_flag'] != 'Yes']

print(new_all_merged)

         user_id  Gender     STATE  Age date_joined  n_dependants fam_status  \
0          26711  Female  Missouri   48    1/1/2017             3    married   
1          26711  Female  Missouri   48    1/1/2017             3    married   
2          26711  Female  Missouri   48    1/1/2017             3    married   
6          26711  Female  Missouri   48    1/1/2017             3    married   
7          26711  Female  Missouri   48    1/1/2017             3    married   
...          ...     ...       ...  ...         ...           ...        ...   
32404813  135902  Female  Missouri   66    4/1/2020             2    married   
32404814  135902  Female  Missouri   66    4/1/2020             2    married   
32404815  135902  Female  Missouri   66    4/1/2020             2    married   
32404816  135902  Female  Missouri   66    4/1/2020             2    married   
32404817  135902  Female  Missouri   66    4/1/2020             2    married   

          income  product_id  \
0      

In [35]:
# export new df
new_all_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'new_all_merged.pkl'))

# 5. Customer profiling

In [3]:
# get latest merged df
new_all_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'new_all_merged.pkl'))

In [5]:
new_all_merged['Age'].describe()

count    2.441488e+07
mean     4.945734e+01
std      1.848532e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: Age, dtype: float64

In [7]:
# create age groups
def age_group(row):

  if row['Age'] < 26:
    return 'young adult'
  elif (row['Age'] >= 26) and (row['Age'] < 40):
    return 'adult'
  elif (row['Age'] >= 40) and (row['Age'] < 60):
    return 'middle adult'
  elif row['Age'] >= 60:
    return 'senior'
  else: return 'Not enough data'

In [9]:
new_all_merged['age_group'] = new_all_merged.apply(age_group, axis=1)

In [11]:
new_all_merged['age_group'].value_counts(dropna=False)

age_group
senior          8330598
middle adult    7665304
adult           5371265
young adult     3047710
Name: count, dtype: int64

In [13]:
# create income groups
def income_group(row):

  if row['income'] < 56600:
    return 'low'
  elif (row['income'] >= 56600) and (row['income'] < 169800):
    return 'middle'
  elif row['income'] >= 169800:
    return 'high'
  else: return 'Not enough data'

In [15]:
new_all_merged['income_group'] = new_all_merged.apply(income_group, axis=1)

In [17]:
new_all_merged['income_group'].value_counts(dropna=False)

income_group
middle    20335470
low        3790160
high        289247
Name: count, dtype: int64

In [19]:
new_all_merged['department_id'].value_counts(dropna=False)

department_id
4     7227091
16    4120488
19    2187121
7     2036447
1     1627614
13    1385375
3      883330
15     787790
20     783011
9      633725
17     539349
14     524945
12     523890
18     335947
11     328480
6      197900
5      107972
8       72111
21      57567
10      27558
2       27166
Name: count, dtype: int64

In [21]:
#label departments
def dept_label(row):

  if row['department_id'] == 1:
    return 'frozen'
  elif (row['department_id'] == 2):
    return 'other'
  elif (row['department_id'] == 3):
    return 'bakery'
  elif (row['department_id'] == 4):
    return 'produce' 
  elif (row['department_id'] == 5):
    return 'alcohol'
  elif (row['department_id'] == 6):
    return 'international'
  elif (row['department_id'] == 7): 
    return 'beverages'
  elif (row['department_id'] == 8):
    return	'pets'
  elif (row['department_id'] == 9):
    return	'dry goods pasta'
  elif (row['department_id'] == 10):
    return	'bulk'
  elif (row['department_id'] == 11):
    return	'personal care'
  elif (row['department_id'] == 12):
    return	'meat seafood'
  elif (row['department_id'] == 13): 
    return 'pantry'
  elif (row['department_id'] == 14): 
    return 'breakfast'
  elif (row['department_id'] == 15): 
    return 'canned goods'
  elif (row['department_id'] == 16): 
    return 'dairy eggs'
  elif (row['department_id'] == 17): 
    return 'household'
  elif (row['department_id'] == 18): 
    return 'babies'
  elif (row['department_id'] == 19): 
    return 'snacks'
  elif (row['department_id'] == 20): 
    return 'deli'
  elif (row['department_id'] == 21): 
    return 'missing'
  else: return 'Not enough data'


In [23]:
new_all_merged['dept_label'] = new_all_merged.apply(dept_label, axis=1)

In [25]:
new_all_merged['dept_label'].value_counts(dropna=False)

dept_label
produce            7227091
dairy eggs         4120488
snacks             2187121
beverages          2036447
frozen             1627614
pantry             1385375
bakery              883330
canned goods        787790
deli                783011
dry goods pasta     633725
household           539349
breakfast           524945
meat seafood        523890
babies              335947
personal care       328480
international       197900
alcohol             107972
pets                 72111
missing              57567
bulk                 27558
other                27166
Name: count, dtype: int64

In [27]:
new_all_merged.groupby('dept_label').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                        mean min max
dept_label                          
alcohol            20.644250   5  99
babies             23.714705   5  99
bakery             21.987157   5  99
beverages          21.951229   5  99
breakfast          21.635206   5  99
bulk               24.707163   5  99
canned goods       21.047112   5  99
dairy eggs         22.384589   5  99
deli               21.277737   5  99
dry goods pasta    20.839194   5  99
frozen             20.311178   5  99
household          20.589620   5  99
international      21.489677   5  99
meat seafood       20.630062   5  99
missing            26.978512   5  99
other              22.258411   5  99
pantry             21.585265   5  99
personal care      21.147711   5  99
pets               19.914645   5  99
produce            22.597392   5  99
snacks             21.891988   5  99

In [29]:
# create dependants flag
new_all_merged.loc[new_all_merged['n_dependants'] >= 1, 'dependents_flag'] = 'Has dependents'

In [31]:
new_all_merged.loc[new_all_merged['n_dependants'] == 0, 'dependents_flag'] = 'No dependents'

In [33]:
new_all_merged['dependents_flag'].value_counts(dropna=False)

dependents_flag
Has dependents    18309875
No dependents      6105002
Name: count, dtype: int64

In [35]:
#single adult #young parent
new_all_merged.groupby('dependents_flag').agg({'Age': ['mean', 'min', 'max']})

Age        
                      mean min max
dependents_flag                   
Has dependents   49.427370  18  81
No dependents    49.547218  18  81

In [69]:
crosstab = pd.crosstab(new_all_merged['dependents_flag'], new_all_merged['dept_label'], dropna = False)

In [70]:
crosstab.to_clipboard()

In [73]:
crosstab = pd.crosstab(new_all_merged['dependents_flag'], new_all_merged['age_group'], dropna = False)

In [74]:
crosstab.to_clipboard()

In [81]:
# create young parent profile
df1 = new_all_merged[new_all_merged['age_group']== 'young adult']
df2 = pd.crosstab(df1['dependents_flag'], df1['price_range_loc'])

In [85]:
mask = new_all_merged['age_group']== 'young adult'
df2 = pd.crosstab(new_all_merged.loc[mask, 'dependents_flag'], new_all_merged.loc[mask, 'price_range_loc'])

In [89]:
df2.head()

price_range_loc,High-range product,Low-range product,Mid-range product
dependents_flag,,,
Has dependents,29348,717948,1552538
No dependents,9349,234818,503709


In [101]:
# create single adult profile
df1 = new_all_merged[new_all_merged['age_group']== 'young adult']
df3 = pd.crosstab(df1['fam_status'], df1['price_range_loc'])

In [103]:
mask = new_all_merged['age_group']== 'young adult'
df3 = pd.crosstab(new_all_merged.loc[mask, 'fam_status'], new_all_merged.loc[mask, 'price_range_loc'])

In [105]:
df3.head()

price_range_loc,High-range product,Low-range product,Mid-range product
fam_status,,,
living with parents and siblings,14771,365097,791531
married,14577,352851,761007
single,9349,234818,503709


In [107]:
# export new df
new_all_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'new_all_merged_plus.pkl'))